In [ ]:
%run conceptors.py

In [ ]:
%pylab inline

In [ ]:

# making some waveform patterns that we might use
pSaw = lambda n: (round(n % waveLengthSamples) / waveLengthSamples * 2) - 1.0
pPulse = lambda n: (((n % waveLengthSamples) < (waveLengthSamples * 0.5)) * 2) - 1.0
pSine2 = lambda n: (np.sin(n) * np.sin((n+np.pi/4)/6))
pSine3 = lambda n: (np.sin(n) * np.sin((n/4)/6)/6)
pJ1 = lambda n: 1 * np.sin(2 * np.pi * n / 3.1504531)
pJ1b = lambda n: 1 * np.sin(n/2) ** 1

period2 = 2
rawp = np.random.randn(period2)
# rawp = np.array([1.1929,2.6856]);
maxVal = np.max(rawp)
minVal = np.min(rawp)
rp = 0.5 * (2 * (rawp - minVal) / (maxVal - minVal) - 1);
pJ2 = lambda n: rp[np.mod(n, period2 )]
pTri = lambda n,p: (((n % p) >= (p/2)) * ((p/2) - (n % (p/2))) + ((n % p) < (p/2)) * (n % (p/2))) * (2/p)



In [ ]:
figsize(20,4)
# plot([pJ1b(x) for x in arange(15)])
# plot([pJ1(x) for x in arange(15)])
# plot([pJ2(x) for x in arange(15)])
plot([pSine2(x) for x in arange(15)])

# plot([pTri(x,10) for x in arange(10)])

In [ ]:
params = {'N':30, 'NetSR':1.6, 'NetinpScaling':1.6,'BiasScaling':0.3,'TychonovAlpha':0.0001,
         'washoutLength':100, 'learnLength':500, 'TychonovAlphaReadout':0.0001,
         'learnLengthWout':500, 'recallTestLength':100,
         'alphas':np.array([12.0,24.0,36.0]),
          'patts':np.array([pJ1b, pJ2,pSine2])
}

net = makeNetwork(params)


In [ ]:
morphValues = [1,0,0]

def onMorph(ind,f):
    global morphValues
#     print("onMorph", f)
    morphValues[ind] = f
    return morphValues

def onExit():
    print( "exiting" )
    global keepRunning
    keepRunning = False
    return True

oscserver = makeOSCServer(onMorph,onExit)

In [ ]:
import time

#run a network
x = 0.5 * np.random.randn(net['p']['N'],1)
keepRunning = True

while keepRunning:
#     pos = morphValue
    C = (net['Cs'][0,0].dot(morphValues[0])) + (net['Cs'][0,1].dot((morphValues[1]))) + (net['Cs'][0,2].dot((morphValues[2])))
    Wsr = net['W'].dot(1.2)
    for i_oversample in range(1):
        x = np.tanh(Wsr.dot(x) + net['Wbias'])
        x = C.dot(x)
    #x_CTestPL[:,n-p['washoutLength'],i_pattern] = x[0:3].T
    output = net['Wout'].dot(x)
#     print( output )
#     print( x.flatten() )
    oscserver.send_value( "/output", output[0][0] )
    oscserver.send_array( "/x", x.flatten() )
    time.sleep(0.01)
    

In [ ]:
#test OSC
try:
    target = Address(57400)
except AddressError as err:
    print (str(err))
    sys.exit()

# send message "/foo/message1" with int, float and string arguments
send(target, "/morph", 0.3)

In [ ]:
a = [ 0, 2, 4]
oscserver.send_array("/morph", a)